In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import joblib

# Load dataset
df = pd.read_csv('/content/balanced_fall_detection_dataset.csv')

df['Magnitude'] = np.sqrt(df['AccelerationX']**2 + df['AccelerationY']**2 + df['AccelerationZ']**2)

label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])

X = df[['AccelerationX', 'AccelerationY', 'AccelerationZ', 'Magnitude']]
y = df['Label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
search = RandomizedSearchCV(xgb, param_distributions=param_dist, n_iter=10, scoring='f1_macro', cv=3, random_state=42, n_jobs=-1)
search.fit(X_train, y_train)

best_model = search.best_estimator_
y_pred = best_model.predict(X_test)

print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

joblib.dump(best_model, 'optimized_fall_detection_model.pkl')
joblib.dump(scaler, 'optimized_scaler.pkl')

print("Optimized model and scaler saved successfully.")



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:35:55] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.79      0.67      2030
           1       0.61      0.76      0.68      1997
           2       0.69      0.66      0.68      2021
           3       0.74      0.63      0.68      2032
           4       0.54      0.34      0.42      2022
           5       0.54      0.52      0.53      2087

    accuracy                           0.62     12189
   macro avg       0.62      0.62      0.61     12189
weighted avg       0.62      0.62      0.61     12189

Confusion Matrix:
 [[1613   91  106   15   20  185]
 [  21 1518  111  119  140   88]
 [ 138  180 1341  225   39   98]
 [  50  259  264 1272  113   74]
 [ 480  277   54   49  684  478]
 [ 468  161   73   32  270 1083]]
Optimized model and scaler saved successfully.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import joblib

df = pd.read_csv('balanced_fall_detection_dataset.csv')

df['Magnitude'] = np.sqrt(df['AccelerationX']**2 + df['AccelerationY']**2 + df['AccelerationZ']**2)
df['AccelerationJerk'] = np.sqrt(np.diff(df['AccelerationX'], prepend=0)**2 + np.diff(df['AccelerationY'], prepend=0)**2 + np.diff(df['AccelerationZ'], prepend=0)**2)

label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])

X = df[['AccelerationX', 'AccelerationY', 'AccelerationZ', 'Magnitude', 'AccelerationJerk']]
y = df['Label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.001, 0.01, 0.05, 0.1],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
search = RandomizedSearchCV(xgb, param_distributions=param_dist, n_iter=10, scoring='f1_macro', cv=3, random_state=42, n_jobs=-1)
search.fit(X_train, y_train)

best_model = search.best_estimator_
y_pred = best_model.predict(X_test)

print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

joblib.dump(best_model, 'optimized_fall_detection_model.pkl')
joblib.dump(scaler, 'optimized_scaler.pkl')

print("Optimized model and scaler saved successfully.")


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:22:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.79      0.76      2030
           1       0.67      0.76      0.71      1997
           2       0.72      0.68      0.70      2021
           3       0.75      0.71      0.73      2032
           4       0.54      0.47      0.50      2022
           5       0.56      0.58      0.57      2087

    accuracy                           0.66     12189
   macro avg       0.66      0.66      0.66     12189
weighted avg       0.66      0.66      0.66     12189

Confusion Matrix:
 [[1604   63   77   17   92  177]
 [   8 1513  102  103  175   96]
 [  91  121 1369  292   56   92]
 [  14  188  240 1442   75   73]
 [ 237  242   45   37  956  505]
 [ 233  137   56   24  431 1206]]
Optimized model and scaler saved successfully.


In [ ]:
!pip install optuna
!pip install lightgbm
import pandas as pd
import numpy as np
import os
import joblib
import optuna

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.impute import SimpleImputer

# ✅ 1️⃣ LOAD DATA (Assuming X and y are loaded properly)
# Example:
data = pd.read_csv('/content/balanced_fall_detection_dataset.csv')
X = data.drop('target', axis=1)
y = data['target']

le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.15, random_state=42
)

imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 300, 800, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 0.3),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 6)
    }
    model = XGBClassifier(**params, random_state=42, eval_metric='mlogloss')
    model.fit(X_resampled, y_resampled)
    y_pred = model.predict(X_test_scaled)
    return classification_report(y_test, y_pred, output_dict=True)['accuracy']

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)
best_params = study.best_params

base_models = [
    ('rf', RandomForestClassifier(n_estimators=500, random_state=42)),
    ('xgb', XGBClassifier(**best_params, random_state=42, eval_metric='mlogloss')),
    ('lgbm', LGBMClassifier(n_estimators=500, random_state=42))
]

stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=RandomForestClassifier(n_estimators=300)
)
stack_model.fit(X_resampled, y_resampled)

y_pred = stack_model.predict(X_test_scaled)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

joblib.dump(stack_model, 'optimized_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(imputer, 'imputer.pkl')
joblib.dump(le, 'label_encoder.pkl')

KeyError: "['target'] not found in axis"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
import joblib

# Load dataset
df = pd.read_csv('balanced_fall_detection_dataset.csv')

# Feature engineering
df['Magnitude'] = np.sqrt(df['AccelerationX']**2 + df['AccelerationY']**2 + df['AccelerationZ']**2)
df['AccelerationJerk'] = np.sqrt(np.diff(df['AccelerationX'], prepend=0)**2 + np.diff(df['AccelerationY'], prepend=0)**2 + np.diff(df['AccelerationZ'], prepend=0)**2)

# Encode labels
label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])

# Define features and target
X = df[['AccelerationX', 'AccelerationY', 'AccelerationZ', 'Magnitude', 'AccelerationJerk']]
y = df['Label']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Create a pipeline with PCA for dimensionality reduction
pipeline = Pipeline([
    ('pca', PCA(n_components=0.95)),  # Retain 95% of variance
    ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'))
])

# Hyperparameter tuning
param_dist = {
    'classifier__n_estimators': [100, 200, 300, 500],
    'classifier__max_depth': [3, 5, 7, 9],
    'classifier__learning_rate': [0.001, 0.01, 0.05, 0.1],
    'classifier__subsample': [0.6, 0.8, 1.0],
    'classifier__colsample_bytree': [0.6, 0.8, 1.0]
}

search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=20, scoring='f1_macro', cv=3, random_state=42, n_jobs=-1)
search.fit(X_train, y_train)

# Evaluate the model
best_model = search.best_estimator_
y_pred = best_model.predict(X_test)

print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Save the model and scaler
joblib.dump(best_model, 'optimized_fall_detection_model.pkl')
joblib.dump(scaler, 'optimized_scaler.pkl')

print("Optimized model and scaler saved successfully.")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:44:20] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.79      0.76      2030
           1       0.65      0.76      0.70      1997
           2       0.70      0.66      0.68      2021
           3       0.73      0.71      0.72      2032
           4       0.53      0.45      0.49      2022
           5       0.56      0.55      0.55      2087

    accuracy                           0.65     12189
   macro avg       0.65      0.65      0.65     12189
weighted avg       0.65      0.65      0.65     12189

Confusion Matrix:
 [[1606   54   83   20  116  151]
 [   8 1517  114  120  148   90]
 [  90  130 1334  320   55   92]
 [  16  194  251 1446   61   64]
 [ 234  266   57   46  909  510]
 [ 241  169   66   36  426 1149]]
Optimized model and scaler saved successfully.


In [ ]:
# Install required libraries (Uncomment if not installed)
# !pip install optuna lightgbm catboost xgboost imbalanced-learn joblib pandas numpy scikit-learn

import pandas as pd
import numpy as np
import optuna
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import BorderlineSMOTE
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

# ✅ 1️⃣ LOAD DATA (Replace with actual dataset)
# data = pd.read_csv('data.csv')
# X = data.drop('target', axis=1)
# y = data['target']

# ✅ Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# ✅ Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.15, random_state=42
)

# ✅ Feature Selection using XGBoost
xgb_temp = XGBClassifier(n_estimators=500, random_state=42)
xgb_temp.fit(X_train, y_train)
feature_importance = xgb_temp.feature_importances_

# Keep only the top 20 most important features
important_features = X.columns[np.argsort(-feature_importance)[:20]]
X_train = X_train[important_features]
X_test = X_test[important_features]

# ✅ Missing Value Handling using KNN Imputer
imputer = KNNImputer(n_neighbors=5)
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# ✅ Robust Scaling (Better than StandardScaler)
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# ✅ Apply PCA for Noise Reduction
pca = PCA(n_components=15)  # Reduce to 15 features
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# ✅ Use Borderline-SMOTE for better data balancing
smote = BorderlineSMOTE(random_state=42, kind='borderline-1')
X_resampled, y_resampled = smote.fit_resample(X_train_pca, y_train)

# ✅ Hyperparameter Tuning with Optuna (LGBM)
def lgbm_objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50)
    }
    model = LGBMClassifier(**params, random_state=42)
    model.fit(X_resampled, y_resampled)
    y_pred = model.predict(X_test_pca)
    return accuracy_score(y_test, y_pred)

lgbm_study = optuna.create_study(direction='maximize')
lgbm_study.optimize(lgbm_objective, n_trials=300)  # Increased trials for better tuning
best_lgbm_params = lgbm_study.best_params

# ✅ Hyperparameter Tuning with Optuna (XGBoost)
def xgb_objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 0.3),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 6)
    }
    model = XGBClassifier(**params, random_state=42, eval_metric='mlogloss')
    model.fit(X_resampled, y_resampled)
    y_pred = model.predict(X_test_pca)
    return accuracy_score(y_test, y_pred)

xgb_study = optuna.create_study(direction='maximize')
xgb_study.optimize(xgb_objective, n_trials=300)
best_xgb_params = xgb_study.best_params

# ✅ Train Final Models with Optimized Parameters
rf_model = RandomForestClassifier(n_estimators=500, random_state=42)
xgb_model = XGBClassifier(**best_xgb_params, random_state=42, eval_metric='mlogloss')
lgbm_model = LGBMClassifier(**best_lgbm_params, random_state=42)
cat_model = CatBoostClassifier(n_estimators=500, depth=8, learning_rate=0.03, verbose=0)

# ✅ Stacking Classifier (Better than Voting Classifier)
meta_learner = LogisticRegression()

stacking_clf = StackingClassifier(
    estimators=[
        ('rf', rf_model),
        ('xgb', xgb_model),
        ('lgbm', lgbm_model),
        ('cat', cat_model)
    ],
    final_estimator=meta_learner
)

# ✅ Train Stacking Classifier
stacking_clf.fit(X_resampled, y_resampled)

# ✅ Evaluate Model
y_pred = stacking_clf.predict(X_test_pca)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# ✅ Save Model and Preprocessing Objects
joblib.dump(stacking_clf, 'final_stacking_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(imputer, 'imputer.pkl')
joblib.dump(pca, 'pca.pkl')
joblib.dump(le, 'label_encoder.pkl')


NameError: name 'y' is not defined

In [ ]:
# Install required libraries (Uncomment if not installed)
# !pip install optuna lightgbm catboost xgboost imbalanced-learn joblib pandas numpy scikit-learn

import pandas as pd
import numpy as np
import optuna
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import BorderlineSMOTE
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

# ✅ 1️⃣ LOAD DATA (Replace with actual dataset)
data = pd.read_csv('balanced_fall_detection_dataset.csv')  # Assuming this is your dataset file
# Assuming 'Label' is your target column
X = data.drop('Label', axis=1)
y = data['Label']  # Define y here

# ✅ Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# ✅ Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.15, random_state=42
)

# ✅ Feature Selection using XGBoost
xgb_temp = XGBClassifier(n_estimators=500, random_state=42)
xgb_temp.fit(X_train, y_train)
feature_importance = xgb_temp.feature_importances_

# Keep only the top 20 most important features
important_features = X.columns[np.argsort(-feature_importance)[:20]]
X_train = X_train[important_features]
X_test = X_test[important_features]

# ✅ Missing Value Handling using KNN Imputer
imputer = KNNImputer(n_neighbors=5)
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# ✅ Robust Scaling (Better than StandardScaler)
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# ✅ Apply PCA for Noise Reduction
max_pca_components = min(15, X_train_scaled.shape[1])  # Use at most 15 or the number of available features
pca = PCA(n_components=max_pca_components)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# ✅ Use Borderline-SMOTE for better data balancing
smote = BorderlineSMOTE(random_state=42, kind='borderline-1')
X_resampled, y_resampled = smote.fit_resample(X_train_pca, y_train)

# ✅ Hyperparameter Tuning with Optuna (LGBM)
def lgbm_objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50)
    }
    model = LGBMClassifier(**params, random_state=42)
    model.fit(X_resampled, y_resampled)
    y_pred = model.predict(X_test_pca)
    return accuracy_score(y_test, y_pred)

lgbm_study = optuna.create_study(direction='maximize')
lgbm_study.optimize(lgbm_objective, n_trials=300)  # Increased trials for better tuning
best_lgbm_params = lgbm_study.best_params

# ✅ Hyperparameter Tuning with Optuna (XGBoost)
def xgb_objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 0.3),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 6)
    }
    model = XGBClassifier(**params, random_state=42, eval_metric='mlogloss')
    model.fit(X_resampled, y_resampled)
    y_pred = model.predict(X_test_pca)
    return accuracy_score(y_test, y_pred)

xgb_study = optuna.create_study(direction='maximize')
xgb_study.optimize(xgb_objective, n_trials=300)
best_xgb_params = xgb_study.best_params

# ✅ Train Final Models with Optimized Parameters
rf_model = RandomForestClassifier(n_estimators=500, random_state=42)
xgb_model = XGBClassifier(**best_xgb_params, random_state=42, eval_metric='mlogloss')
lgbm_model = LGBMClassifier(**best_lgbm_params, random_state=42)
cat_model = CatBoostClassifier(n_estimators=500, depth=8, learning_rate=0.03, verbose=0)

# ✅ Stacking Classifier (Better than Voting Classifier)
meta_learner = LogisticRegression()

stacking_clf = StackingClassifier(
    estimators=[
        ('rf', rf_model),
        ('xgb', xgb_model),
        ('lgbm', lgbm_model),
        ('cat', cat_model)
    ],
    final_estimator=meta_learner
)

# ✅ Train Stacking Classifier
stacking_clf.fit(X_resampled, y_resampled)

# ✅ Evaluate Model
y_pred = stacking_clf.predict(X_test_pca)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# ✅ Save Model and Preprocessing Objects
joblib.dump(stacking_clf, 'final_stacking_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(imputer, 'imputer.pkl')
joblib.dump(pca, 'pca.pkl')
joblib.dump(le, 'label_encoder.pkl')

[I 2025-03-13 04:37:50,562] A new study created in memory with name: no-name-4db1d770-7bdc-4b3f-b005-421ab9a1c970
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002512 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 51966, number of used features: 3
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-03-13 04:38:24,702] Trial 0 finished with value: 0.5818201706409976 and parameters: {'num_leaves': 87, 'learning_rate': 0.026964749121822354, 'n_estimators': 800, 'subsample': 0.731168357550096, 'colsample_bytree': 0.7727387002825166, 'min_child_samples': 25}. Best is trial 0 with value: 0.5818201706409976.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001778 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 51966, number of used features: 3
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-03-13 04:38:30,646] Trial 1 finished with value: 0.6126668125136732 and parameters: {'num_leaves': 42, 'learning_rate': 0.07627804749240272, 'n_estimators': 200, 'subsample': 0.6118131025017626, 'colsample_bytree': 0.9830761906187663, 'min_child_samples': 38}. Best is trial 1 with value: 0.6126668125136732.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-03-13 06:11:47,219] Trial 297 finished with value: 0.6186830015313936 and parameters: {'num_leaves': 127, 'learning_rate': 0.008697734107257543, 'n_estimators': 300, 'subsample': 0.8024227945745975, 'colsample_bytree': 0.9721989491025373, 'min_child_samples': 12}. Best is trial 163 with value: 0.6217457886676876.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001741 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 51966, number of used features: 3
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-03-13 06:12:09,417] Trial 298 finished with value: 0.6179173047473201 and parameters: {'num_leaves': 137, 'learning_rate': 0.014513188325944454, 'n_estimators': 400, 'subsample': 0.9161193301351968, 'colsample_bytree': 0.9489502154854677, 'min_child_samples': 8}. Best is trial 163 with value: 0.6217457886676876.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002942 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 51966, number of used features: 3
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-03-13 06:12:27,920] Trial 299 finished with value: 0.6186830015313936 and parameters: {'num_leaves': 141, 'learning_rate': 0.007040074251931813, 'n_estimators': 300, 'subsample': 0.9524862747444465, 'colsample_bytree': 0.9807910148544535, 'min_child_samples': 11}. Best is trial 163 with value: 0.6217457886676876.
[I 2025-03-13 06:12:27,924] A new study created in memory with name: no-name-97c0d455-e493-4a3c-aead-3cccceef8156
[I 2025-03-13 06:12:55,620] Trial 0 finished with value: 0.600415663968497 and parameters: {'n_estimators': 800, 'max_depth': 6, 'learning_rate': 0.017418569473918087, 'subsample': 0.8879430298002999, 'colsample_bytree': 0.7029981147965911, 'gamma': 0.2914200179079851, 'min_child_weight': 3}. Best is trial 0 with value: 0.600415663968497.
[I 2025-03-13 06:13:17,155]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPooling1D, Flatten, BatchNormalization

# Load dataset
df = pd.read_csv("/content/cleaned_healthcare_dataset.csv")
df.drop(columns=['id'], inplace=True)  # Remove ID column

# Encoding categorical variables
encoder = LabelEncoder()
categorical_cols = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])

# Normalize numerical features
scaler = StandardScaler()
numerical_cols = ['age', 'avg_glucose_level', 'bmi']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Feature selection for sensor data (Assuming sensor columns exist)
sensor_cols = ['ECG', 'PPG', 'Acc_X', 'Acc_Y', 'Acc_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z', 'EMG']
X = df[sensor_cols] if all(col in df.columns for col in sensor_cols) else df.drop(columns=['stroke'])
y = df['stroke']

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Reshape for LSTM if time-series data is available
X_resampled = np.array(X_resampled).reshape((X_resampled.shape[0], X_resampled.shape[1], 1))

# Splitting into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Build CNN-LSTM Model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train Model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Save model for later use
model.save("stroke_prediction_model.h5")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.7349 - loss: 0.5266 - val_accuracy: 0.7604 - val_loss: 0.4905
Epoch 2/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7939 - loss: 0.4341 - val_accuracy: 0.8026 - val_loss: 0.4133
Epoch 3/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8115 - loss: 0.4036 - val_accuracy: 0.8180 - val_loss: 0.3893
Epoch 4/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8240 - loss: 0.3821 - val_accuracy: 0.8365 - val_loss: 0.3716
Epoch 5/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8336 - loss: 0.3673 - val_accuracy: 0.8468 - val_loss: 0.3619
Epoch 6/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8413 - loss: 0.3653 - val_accuracy: 0.8540 - val_loss: 0.3389
Epoch 7/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8595 - loss: 0.3310 - val_accuracy: 0.8581 - val_loss: 0.3273
Epoch 8/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8715 - loss: 0.3036 - val_accuracy: 

Test Accuracy: 92.75%


In [ ]:
# Install required libraries (Uncomment if not installed)
# !pip install optuna lightgbm catboost xgboost imbalanced-learn joblib pandas numpy scikit-learn
!pip install optuna
!pip install catboost
import pandas as pd
import numpy as np
import optuna
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import BorderlineSMOTE
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

# ✅ 1️⃣ LOAD DATA (Replace with actual dataset)
data = pd.read_csv('/balanced_fall_detection_dataset.csv')  # Assuming this is your dataset file

# Assuming 'Label' is your target column
X = data.drop('Label', axis=1)
y = data['Label']  # Define y here

# ✅ Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# ✅ Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.15, random_state=42
)

# ✅ Feature Selection using XGBoost
xgb_temp = XGBClassifier(n_estimators=500, random_state=42)
xgb_temp.fit(X_train, y_train)
feature_importance = xgb_temp.feature_importances_

# Keep only the top 20 most important features
important_features = X.columns[np.argsort(-feature_importance)[:20]]
X_train = X_train[important_features]
X_test = X_test[important_features]

# ✅ Missing Value Handling using KNN Imputer
imputer = KNNImputer(n_neighbors=5)
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# ✅ Robust Scaling (Better than StandardScaler)
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# ✅ Apply PCA for Noise Reduction
max_pca_components = min(15, X_train_scaled.shape[1])  # Ensure PCA does not exceed feature count
pca = PCA(n_components=max_pca_components)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# ✅ Use Borderline-SMOTE for better data balancing
smote = BorderlineSMOTE(random_state=42, kind='borderline-1')
X_resampled, y_resampled = smote.fit_resample(X_train_pca, y_train)

# ✅ Hyperparameter Tuning with Optuna (LGBM)
def lgbm_objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50)
    }
    model = LGBMClassifier(**params, random_state=42)
    model.fit(X_resampled, y_resampled)
    y_pred = model.predict(X_test_pca)
    return accuracy_score(y_test, y_pred)

lgbm_study = optuna.create_study(direction='maximize')
lgbm_study.optimize(lgbm_objective, n_trials=300)  # Increased trials for better tuning
best_lgbm_params = lgbm_study.best_params

# ✅ Hyperparameter Tuning with Optuna (XGBoost)
def xgb_objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 0.3),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 6)
    }
    model = XGBClassifier(**params, random_state=42, eval_metric='mlogloss')
    model.fit(X_resampled, y_resampled)
    y_pred = model.predict(X_test_pca)
    return accuracy_score(y_test, y_pred)

xgb_study = optuna.create_study(direction='maximize')
xgb_study.optimize(xgb_objective, n_trials=300)
best_xgb_params = xgb_study.best_params

# ✅ Train Final Models with Optimized Parameters
rf_model = RandomForestClassifier(n_estimators=500, random_state=42)
xgb_model = XGBClassifier(**best_xgb_params, random_state=42, eval_metric='mlogloss')
lgbm_model = LGBMClassifier(**best_lgbm_params, random_state=42)
cat_model = CatBoostClassifier(n_estimators=500, depth=8, learning_rate=0.03, verbose=0)

# ✅ Stacking Classifier (Better than Voting Classifier)
meta_learner = LogisticRegression()

stacking_clf = StackingClassifier(
    estimators=[
        ('rf', rf_model),
        ('xgb', xgb_model),
        ('lgbm', lgbm_model),
        ('cat', cat_model)
    ],
    final_estimator=meta_learner
)

# ✅ Train Stacking Classifier
stacking_clf.fit(X_resampled, y_resampled)

# ✅ Evaluate Model
y_pred = stacking_clf.predict(X_test_pca)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))

# ✅ Save Model and Preprocessing Objects
joblib.dump(stacking_clf, 'final_stacking_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(imputer, 'imputer.pkl')
joblib.dump(pca, 'pca.pkl')
joblib.dump(le, 'label_encoder.pkl')


[I 2025-03-13 17:32:43,542] A new study created in memory with name: no-name-ff096e77-4955-42d1-a346-babe9b509a5e
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002055 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 51966, number of used features: 3
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-03-13 17:33:11,488] Trial 0 finished with value: 0.5899146795012032 and parameters: {'num_leaves': 94, 'learning_rate': 0.013059793370222367, 'n_estimators': 600, 'subsample': 0.6989421290966229, 'colsample_bytree': 0.6534394407258914, 'min_child_samples': 50}. Best is trial 0 with value: 0.5899146795012032.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001709 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 51966, number of used features: 3
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-03-13 17:33:40,749] Trial 1 finished with value: 0.6141982060818202 and parameters: {'num_leaves': 76, 'learning_rate': 0.010073678862368696, 'n_estimators': 700, 'subsample': 0.7070510361909101, 'colsample_bytree': 0.9923848637862408, 'min_child_samples': 13}. Best is trial 1 with value: 0.6141982060818202.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001406 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 41573, number of used features: 3
[LightGBM] [Info] Start training from score -1.791735
[LightGBM] [Info] Start training from score -1.791735
[LightGBM] [Info] Start training from score -1.791880
[LightGBM] [Info] Start training from score -1.791735
[LightGBM] [Info] Start training from score -1.791735
[LightGBM] [Info] Start training from score -1.791735


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.74      0.66      1518
           1       0.64      0.75      0.69      1506
           2       0.69      0.67      0.68      1530
           3       0.72      0.63      0.67      1496
           4       0.52      0.40      0.45      1521
           5       0.52      0.52      0.52      1571

    accuracy                           0.62      9142
   macro avg       0.62      0.62      0.61      9142
weighted avg       0.61      0.62      0.61      9142

Confusion Matrix:
 [[1124   56   92   12   69  165]
 [  13 1122   77  100  116   78]
 [ 111  119 1025  168   31   76]
 [  34  181  212  937   80   52]
 [ 295  175   28   44  605  374]
 [ 303  111   54   32  254  817]]
Accuracy Score: 0.6158389849048348


['label_encoder.pkl']

In [ ]:
# Install missing libraries if necessary
# !pip install optuna lightgbm catboost xgboost imbalanced-learn joblib pandas numpy scikit-learn
!pip install optuna
!pip install catboost
import pandas as pd
import numpy as np
import optuna
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import BorderlineSMOTE, ADASYN
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

# ✅ Load Dataset
data = pd.read_csv('/balanced_fall_detection_dataset.csv')
X = data.drop('Label', axis=1)
y = data['Label']

# ✅ Encode Labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# ✅ Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.15, random_state=42)

# ✅ Feature Selection: Correlation + XGBoost Importance
xgb_temp = XGBClassifier(n_estimators=500, random_state=42)
xgb_temp.fit(X_train, y_train)
feature_importance = xgb_temp.feature_importances_

important_features = X.columns[np.argsort(-feature_importance)[:30]]  # Increased to 30 features
X_train = X_train[important_features]
X_test = X_test[important_features]

# ✅ Handle Missing Data with KNN Imputer
imputer = KNNImputer(n_neighbors=5)
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# ✅ Robust Scaling
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# ✅ Dimensionality Reduction (PCA Alternative)
max_pca_components = min(10, X_train_scaled.shape[1])
pca = PCA(n_components=max_pca_components)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# ✅ Hybrid Data Resampling (SMOTE + ADASYN)
smote = BorderlineSMOTE(random_state=42, kind='borderline-1')
adasyn = ADASYN(random_state=42, sampling_strategy='minority')

X_resampled, y_resampled = smote.fit_resample(X_train_pca, y_train)
X_resampled, y_resampled = adasyn.fit_resample(X_resampled, y_resampled)

# ✅ Optuna Hyperparameter Tuning (LGBM)
def lgbm_objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 20, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 200, 2000, step=100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100)
    }
    model = LGBMClassifier(**params, random_state=42)
    model.fit(X_resampled, y_resampled)
    y_pred = model.predict(X_test_pca)
    return accuracy_score(y_test, y_pred)

lgbm_study = optuna.create_study(direction='maximize')
lgbm_study.optimize(lgbm_objective, n_trials=500)  # More trials for better tuning
best_lgbm_params = lgbm_study.best_params

# ✅ Optuna Hyperparameter Tuning (XGBoost)
def xgb_objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 2000, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 0.3),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
    }
    model = XGBClassifier(**params, random_state=42, eval_metric='mlogloss')
    model.fit(X_resampled, y_resampled)
    y_pred = model.predict(X_test_pca)
    return accuracy_score(y_test, y_pred)

xgb_study = optuna.create_study(direction='maximize')
xgb_study.optimize(xgb_objective, n_trials=500)
best_xgb_params = xgb_study.best_params

# ✅ Train Final Models
rf_model = RandomForestClassifier(n_estimators=700, random_state=42)
xgb_model = XGBClassifier(**best_xgb_params, random_state=42, eval_metric='mlogloss')
lgbm_model = LGBMClassifier(**best_lgbm_params, random_state=42)
cat_model = CatBoostClassifier(n_estimators=700, depth=10, learning_rate=0.02, verbose=0)

# ✅ Optimized Stacking Model
meta_learner = LogisticRegression(penalty='l1', solver='liblinear')

stacking_clf = StackingClassifier(
    estimators=[
        ('rf', rf_model),
        ('xgb', xgb_model),
        ('lgbm', lgbm_model),
        ('cat', cat_model)
    ],
    final_estimator=meta_learner
)

# ✅ Train Stacking Classifier
stacking_clf.fit(X_resampled, y_resampled)

# ✅ Evaluate Model
y_pred = stacking_clf.predict(X_test_pca)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

# ✅ Save Model and Preprocessing Objects
joblib.dump(stacking_clf, 'final_stacking_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(imputer, 'imputer.pkl')
joblib.dump(pca, 'pca.pkl')
joblib.dump(le, 'label_encoder.pkl')


[I 2025-03-14 05:27:22,752] A new study created in memory with name: no-name-0b6ed93a-a054-4ec1-a13a-6b54fe557349
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001914 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 51966, number of used features: 3
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-03-14 05:28:09,208] Trial 0 finished with value: 0.5544738569240867 and parameters: {'num_leaves': 85, 'learning_rate': 0.05053973983080206, 'n_estimators': 1500, 'subsample': 0.6104769660045817, 'colsample_bytree': 0.7235065927015019, 'min_child_samples': 8}. Best is trial 0 with value: 0.5544738569240867.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002992 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 51966, number of used features: 3
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-03-14 05:28:18,977] Trial 1 finished with value: 0.5973528768322031 and parameters: {'num_leaves': 31, 'learning_rate': 0.006594068297712214, 'n_estimators': 300, 'subsample': 0.8298349347409999, 'colsample_bytree': 0.8822782356621104, 'min_child_samples': 68}. Best is trial 1 with value: 0.5973528768322031.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001679 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 51966, number of used features: 3
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-03-14 06:54:41,411] Trial 77 finished with value: 0.6175891489827171 and parameters: {'num_leaves': 146, 'learning_rate': 0.0015533214635587427, 'n_estimators': 1600, 'subsample': 0.7643899959200308, 'colsample_bytree': 0.8509090954515869, 'min_child_samples': 53}. Best is trial 64 with value: 0.6199956245898053.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001883 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 51966, number of used features: 3
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-03-14 06:56:45,615] Trial 78 finished with value: 0.6170422227083789 and parameters: {'num_leaves': 151, 'learning_rate': 0.001066796144887112, 'n_estimators': 1800, 'subsample': 0.9285864268576812, 'colsample_bytree': 0.8590103203783082, 'min_child_samples': 42}. Best is trial 64 with value: 0.6199956245898053.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001732 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 51966, number of used features: 3
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load dataset
file_path = "/content/stroke_detection_dataset (1).csv"
df = pd.read_csv(file_path)

# Split features and labels
X = df.drop(columns=['Label']).values  # Sensor data
y = df['Label'].values  # Stroke (1) or Fake Stroke (0)

# Normalize sensor data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape for LSTM (samples, timesteps, features)
X_reshaped = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

# Build LSTM Model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(1, X_scaled.shape[1])),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test))

# Evaluate accuracy
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Save model for ESP32 TensorFlow Lite conversion
model.save("/mnt/data/stroke_detection_model.h5")
print("Model saved successfully!")



Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.6624 - loss: 0.6536 - val_accuracy: 0.6880 - val_loss: 0.6210
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6756 - loss: 0.6333 - val_accuracy: 0.6880 - val_loss: 0.6218
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6798 - loss: 0.6287 - val_accuracy: 0.6880 - val_loss: 0.6217
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6723 - loss: 0.6315 - val_accuracy: 0.6880 - val_loss: 0.6211
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7093 - loss: 0.6058 - val_accuracy: 0.6880 - val_loss: 0.6229
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6925 - loss: 0.6178 - val_accuracy: 0.6880 - val_loss: 0.6224
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6885 - loss: 0.6165 - val_accuracy: 0.6880 - val_loss: 0.6229
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6965 - loss: 0.6084 - val_accuracy: 0.6880 - val_

Model Accuracy: 67.00%
Model saved successfully!


In [ ]:
!pip install tensorflow numpy pandas scikit-learn imbalanced-learn
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, Flatten, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE

# ✅ Load dataset
df = pd.read_csv("/content/stroke_prediction_dataset_58000_real_fake.csv")

# ✅ Separate features (X) and labels (y)
X = df.iloc[:, :-1].values  # Sensor data (features)
y = df.iloc[:, -1].values   # Stroke classification (1: Real, -1: Fake, 0: No Stroke)

# ✅ Convert labels (-1, 0, 1) to (0, 1, 2) for categorical cross-entropy
y = np.where(y == -1, 0, y)  # -1 → 0 (Fake), 0 → 1 (No Stroke), 1 → 2 (Real Stroke)

# ✅ Normalize sensor data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ✅ Handle class imbalance using SMOTE
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# ✅ Reshape input for CNN-LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# ✅ Define CNN-LSTM Model
model = Sequential()

# 🎯 CNN Layers (Feature Extraction)
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Flatten())

# 🎯 LSTM Layers (Time-Series Analysis)
model.add(tf.keras.layers.Reshape((X_train.shape[1], 1)))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(units=32, return_sequences=False))
model.add(Dropout(0.3))

# 🎯 Fully Connected Layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))  # 3 Classes: Fake Stroke (0), No Stroke (1), Real Stroke (2)

# ✅ Compile Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# ✅ Compute class weights for imbalanced data
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# ✅ Train model
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test),
                    class_weight=class_weight_dict, verbose=1)

# ✅ Evaluate Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"🔥 Test Accuracy: {test_acc * 100:.2f}%")

# ✅ Save the Model
model.save("stroke_detection_model.h5")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: The total size of the tensor must be unchanged. Received: input_shape=(768,), target_shape=(10, 1)

In [ ]:
!pip install tensorflow numpy pandas scikit-learn imbalanced-learn
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, Flatten, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE

# ✅ Load dataset
df = pd.read_csv("/content/stroke_prediction_dataset_58000_real_fake.csv")

# ✅ Separate features (X) and labels (y)
X = df.iloc[:, :-1].values  # Sensor data (features)
y = df.iloc[:, -1].values   # Stroke classification (1: Real, -1: Fake, 0: No Stroke)

# ✅ Convert labels (-1, 0, 1) to (0, 1, 2) for categorical cross-entropy
y = np.where(y == -1, 0, y)  # -1 → 0 (Fake), 0 → 1 (No Stroke), 1 → 2 (Real Stroke)

# ✅ Normalize sensor data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ✅ Handle class imbalance using SMOTE
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# ✅ Reshape input for CNN-LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)  # Reshape to (samples, timesteps, features)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# ✅ Define CNN-LSTM Model
model = Sequential()

# 🎯 CNN Layers (Feature Extraction)
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# 🎯 LSTM Layers (Time-Series Analysis)
model.add(LSTM(units=64, return_sequences=True))  # Keep return_sequences=True for multiple LSTMs
model.add(Dropout(0.3))
model.add(LSTM(units=32, return_sequences=False))  # Last LSTM layer should have return_sequences=False
model.add(Dropout(0.3))

# 🎯 Fully Connected Layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))  # 3 Classes: Fake Stroke (0), No Stroke (1), Real Stroke (2)

# ✅ Compile Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# ✅ Compute class weights for imbalanced data
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# ✅ Train model
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test),
                    class_weight=class_weight_dict, verbose=1)

# ✅ Evaluate Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"🔥 Test Accuracy: {test_acc * 100:.2f}%")

# ✅ Save the Model
model.save("stroke_detection_model.h5")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 27s 21ms/step - accuracy: 0.5018 - loss: 0.7604 - val_accuracy: 0.5072 - val_loss: 0.6934
Epoch 2/100
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 41s 20ms/step - accuracy: 0.5087 - loss: 0.6951 - val_accuracy: 0.5189 - val_loss: 0.6924
Epoch 3/100
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy: 0.5165 - loss: 0.6929 - val_accuracy: 0.5225 - val_loss: 0.6911
Epoch 4/100
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 40s 20ms/step - accuracy: 0.5271 - loss: 0.6913 - val_accuracy: 0.5125 - val_loss: 0.6921
Epoch 5/100
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - accuracy: 0.5254 - loss: 0.6915 - val_accuracy: 0.5391 - val_loss: 0.6880
Epoch 6/100
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.5355 - loss: 0.6898 - val_accuracy: 0.5465 - val_loss: 0.6876
Epoch 7/100
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.5387 - loss: 0.6895 - val_accuracy: 0.5466 - val_loss: 0.6867
Epoch 8/100
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 41s 20ms/step - accuracy: 

🔥 Test Accuracy: 60.43%


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE

# ✅ Load dataset
df = pd.read_csv("/content/stroke_prediction_dataset_58000_real_fake.csv")

# ✅ Separate features (X) and labels (y)
X = df.iloc[:, :-1].values  # Sensor data (features)
y = df.iloc[:, -1].values   # Stroke classification (1: Real, -1: Fake, 0: No Stroke)

# ✅ Convert labels (-1, 0, 1) to (0, 1, 2) for categorical classification
y = np.where(y == -1, 0, y)  # -1 → 0 (Fake), 0 → 1 (No Stroke), 1 → 2 (Real Stroke)

# ✅ Normalize sensor data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ✅ Handle class imbalance using SMOTE
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# ✅ Reshape input for CNN-LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# ✅ Define High-Accuracy CNN-BiLSTM Model
model = Sequential()

# 🎯 CNN Layers (Feature Extraction)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# 🎯 BiLSTM Layers (Time-Series Analysis)
model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(units=64, return_sequences=False)))
model.add(Dropout(0.3))

# 🎯 Fully Connected Layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))  # 3 Classes: Fake Stroke (0), No Stroke (1), Real Stroke (2)

# ✅ Compile Model with Focal Loss (Better for Class Imbalance)
def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=3)
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        loss = -alpha * (1 - pt) ** gamma * tf.math.log(pt)
        return tf.reduce_sum(loss, axis=1)
    return loss

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=focal_loss(),
              metrics=['accuracy'])

# ✅ Compute class weights for imbalanced data
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# ✅ Train model with Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test),
                    class_weight=class_weight_dict, callbacks=[early_stopping], verbose=1)

# ✅ Evaluate Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"🔥 Test Accuracy: {test_acc * 100:.2f}%")

# ✅ Save the Model
model.save("stroke_detection_high_accuracy.h5")




/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 64s 54ms/step - accuracy: 0.4958 - loss: 0.1028 - val_accuracy: 0.4965 - val_loss: 0.0876
Epoch 2/200
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 54s 54ms/step - accuracy: 0.5069 - loss: 0.0876 - val_accuracy: 0.4992 - val_loss: 0.0868
Epoch 3/200
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 82s 53ms/step - accuracy: 0.5110 - loss: 0.0870 - val_accuracy: 0.5060 - val_loss: 0.0867
Epoch 4/200
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 82s 53ms/step - accuracy: 0.5187 - loss: 0.0867 - val_accuracy: 0.4955 - val_loss: 0.0872
Epoch 5/200
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 55s 54ms/step - accuracy: 0.5196 - loss: 0.0866 - val_accuracy: 0.5010 - val_loss: 0.0868
Epoch 6/200
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 83s 55ms/step - accuracy: 0.5196 - loss: 0.0866 - val_accuracy: 0.5150 - val_loss: 0.0865
Epoch 7/200
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 80s 53ms/step - accuracy: 0.5243 - loss: 0.0865 - val_accuracy: 0.5126 - val_loss: 0.0866
Epoch 8/200
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 84s 55ms/step - accuracy: 

🔥 Test Accuracy: 63.41%


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# ✅ Load dataset
file_path = "/content/stroke_prediction_dataset_58000_real_fake.csv"
df = pd.read_csv(file_path)

# ✅ Split Features & Labels
X = df.drop(columns=["stroke_label"]).values
y = df["stroke_label"].values  # 0 = No Stroke, 1 = Stroke

# ✅ Standardization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ✅ Handle Class Imbalance using SMOTE
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# ✅ Reshape for LSTM (samples, time_steps, features)
X_resampled = X_resampled.reshape(X_resampled.shape[0], X_resampled.shape[1], 1)

# ✅ Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# ✅ Build Model (BiLSTM + CNN)
model = Sequential()

# 🔹 1D CNN for Feature Extraction
model.add(Conv1D(filters=64, kernel_size=3, activation="relu", input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())

# 🔹 BiLSTM for Temporal Feature Learning
model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(units=64, return_sequences=False)))
model.add(Dropout(0.3))

# 🔹 Dense Layers for Classification
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(32, activation="relu"))
model.add(Dense(1, activation="sigmoid"))  # Output Layer

# ✅ Compile Model
model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

# ✅ Callbacks (Early Stopping & Learning Rate Reduction)
callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
]

# ✅ Train Model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

# ✅ Evaluate Model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# ✅ Save Model
model.save("/mnt/data/stroke_detection_model.h5")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1081/1081 ━━━━━━━━━━━━━━━━━━━━ 54s 42ms/step - accuracy: 0.3350 - loss: -0.0651 - val_accuracy: 0.3312 - val_loss: -0.1160 - learning_rate: 0.0010
Epoch 2/50
1081/1081 ━━━━━━━━━━━━━━━━━━━━ 41s 38ms/step - accuracy: 0.3329 - loss: -0.6168 - val_accuracy: 0.3312 - val_loss: -8.3586 - learning_rate: 0.0010
Epoch 3/50
1081/1081 ━━━━━━━━━━━━━━━━━━━━ 97s 52ms/step - accuracy: 0.3355 - loss: -23.7684 - val_accuracy: 0.3312 - val_loss: -110.1920 - learning_rate: 0.0010
Epoch 4/50
 757/1081 ━━━━━━━━━━━━━━━━━━━━ 13s 41ms/step - accuracy: 0.3324 - loss: -346.7178

KeyboardInterrupt: 

In [ ]:
!pip install pandas numpy tensorflow imbalanced-learn scikit-learn
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, Dropout, BatchNormalization, Conv1D, LSTM, Bidirectional, Flatten,
    GlobalAveragePooling1D, Attention
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, accuracy_score

# Load reduced dataset
file_path = ""
df = pd.read_csv(file_path)

# 🔹 Drop irrelevant columns if any
df.dropna(inplace=True)  # Remove missing values
df.drop_duplicates(inplace=True)  # Remove duplicate rows

# 🔹 Split features and labels
X = df.drop(columns=["stroke"]).values  # Features
y = df["stroke"].values  # Labels (0 = No Stroke, 1 = Real Stroke, 2 = Fake Stroke)

# 🔹 Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ✅ Handle class imbalance using SMOTE
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)
# 🔹 Define Input Shape
input_layer = Input(shape=(X_train.shape[1], 1))

# 🔹 CNN Layers (Feature Extraction)
x = Conv1D(filters=128, kernel_size=3, activation="relu", padding="same")(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# 🔹 BiLSTM Layers (Bidirectional for Better Feature Extraction)
x = Bidirectional(LSTM(units=128, return_sequences=True))(x)
x = Dropout(0.4)(x)

# 🔹 Attention Mechanism (Focus on Important Features)
attention = Attention()([x, x])
x = GlobalAveragePooling1D()(attention)

# 🔹 Fully Connected Layers
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.2)(x)

# 🔹 Output Layer (3 Classes: No Stroke, Real Stroke, Fake Stroke)
output_layer = Dense(3, activation="softmax")(x)

# ✅ Compile Model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# ✅ Model Summary
model.summary()
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,  # Can increase if needed
    batch_size=64,
    verbose=1
)
# Predict on test data
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Print accuracy and classification report
accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

print("\nClassification Report:\n", classification_report(y_test, y_pred_classes))


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import time

# Step 1: Simulate 100 Rows for Training
np.random.seed(42)
n_rows = 100
data = {
    'accel_x': np.random.uniform(-2, 2, n_rows),
    'accel_y': np.random.uniform(-2, 2, n_rows),
    'accel_z': np.random.uniform(-2, 2, n_rows),
    'gyro_x': np.random.uniform(-250, 250, n_rows),
    'gyro_y': np.random.uniform(-250, 250, n_rows),
    'gyro_z': np.random.uniform(-250, 250, n_rows),
    'heart_rate': np.random.uniform(60, 100, n_rows),
    'spo2': np.random.uniform(90, 100, n_rows),
    'emg_signal': np.random.uniform(0.5, 3, n_rows),
    'ecg_signal': np.random.uniform(-1, 1, n_rows),
    'stroke_label': np.concatenate([np.zeros(33), np.ones(33), -np.ones(34)])
}
df_sample = pd.DataFrame(data)

for i in range(n_rows):
    label = df_sample.loc[i, 'stroke_label']
    if label == 1:  # True Stroke
        df_sample.loc[i, 'accel_x'] += np.random.uniform(1, 3)
        df_sample.loc[i, 'gyro_x'] += np.random.uniform(100, 200)
        df_sample.loc[i, 'heart_rate'] = np.random.uniform(100, 140)
        df_sample.loc[i, 'spo2'] = np.random.uniform(85, 92)
        df_sample.loc[i, 'emg_signal'] = np.random.uniform(3, 5)
        df_sample.loc[i, 'ecg_signal'] = np.random.uniform(-2, 2)
    elif label == -1:  # Fake Stroke
        df_sample.loc[i, 'accel_x'] += np.random.uniform(-0.5, 0.5)
        df_sample.loc[i, 'gyro_x'] += np.random.uniform(-50, 50)
        df_sample.loc[i, 'heart_rate'] = np.random.uniform(60, 120)
        df_sample.loc[i, 'spo2'] = np.random.uniform(88, 98)
        df_sample.loc[i, 'emg_signal'] = np.random.uniform(0.2, 1.5)
        df_sample.loc[i, 'ecg_signal'] = np.random.uniform(-0.5, 0.5)

df_sample = df_sample.sample(frac=1).reset_index(drop=True)
df_sample['timestamp'] = np.arange(0, n_rows / 100, 1 / 100)

# Scale to 20,000 samples
n_samples = 20000
repeat_factor = n_samples // len(df_sample) + 1
df = pd.concat([df_sample] * repeat_factor, ignore_index=True).iloc[:n_samples]

for col in df.columns[:-1]:
    df[col] += np.random.normal(0, 0.01, n_samples)

labels = np.array([0, 1, -1])
target_counts = n_samples // 3
for label in labels:
    current_count = (df['stroke_label'] == label).sum()
    if current_count < target_counts:
        to_add = target_counts - current_count
        indices = df[df['stroke_label'] != label].sample(to_add, replace=True).index
        df.loc[indices, 'stroke_label'] = label

df['timestamp'] = np.arange(0, n_samples / 100, 1 / 100)

# Step 2: Preprocessing
window_size = 500
step_size = 250

scaler = StandardScaler()
sensor_cols = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z',
               'heart_rate', 'spo2', 'emg_signal', 'ecg_signal']
df[sensor_cols] = scaler.fit_transform(df[sensor_cols])

def create_windows(df, window_size, step_size):
    X, y = [], []
    for i in range(0, len(df) - window_size + 1, step_size):
        window = df.iloc[i:i + window_size]
        X.append(window[sensor_cols].values)
        y.append(np.bincount(window['stroke_label'].astype(int) + 1).argmax() - 1)
    return np.array(X), np.array(y)

X, y = create_windows(df, window_size, step_size)
print("Windowed data shape:", X.shape, y.shape)

y = tf.keras.utils.to_categorical(y + 1, num_classes=3)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

def augment_data(X):
    noise = np.random.normal(0, 0.05, X.shape)
    return X + noise

X_train_aug = augment_data(X_train)

# Step 3: Model Architecture
def build_model(window_size, num_features):
    mpu_input = layers.Input(shape=(window_size, 6))
    max_input = layers.Input(shape=(window_size, 2))
    emg_input = layers.Input(shape=(window_size, 1))
    ecg_input = layers.Input(shape=(window_size, 1))

    def cnn_branch(x, filters):
        x = layers.Conv1D(filters, 5, activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv1D(filters, 5, activation='relu', padding='same')(x)
        x = layers.MaxPooling1D(2)(x)
        return x

    mpu_features = cnn_branch(mpu_input, 128)
    max_features = cnn_branch(max_input, 64)
    emg_features = cnn_branch(emg_input, 64)
    ecg_features = cnn_branch(ecg_input, 64)

    combined = layers.Concatenate()([mpu_features, max_features, emg_features, ecg_features])
    lstm_out = layers.Bidirectional(layers.LSTM(256, return_sequences=True))(combined)
    attention = layers.Attention()([lstm_out, lstm_out])
    lstm_out = layers.LSTM(128)(attention)
    lstm_out = layers.Dropout(0.5)(lstm_out)

    dense_out = layers.Dense(256, activation='relu')(lstm_out)
    dense_out = layers.Dropout(0.4)(dense_out)
    output = layers.Dense(3, activation='softmax')(dense_out)

    model = Model(inputs=[mpu_input, max_input, emg_input, ecg_input], outputs=output)
    return model

# Prepare training inputs
X_train_mpu = X_train[:, :, :6]
X_train_max = X_train[:, :, 6:8]
X_train_emg = X_train[:, :, 8:9]
X_train_ecg = X_train[:, :, 9:10]

X_val_mpu = X_val[:, :, :6]
X_val_max = X_val[:, :, 6:8]
X_val_emg = X_val[:, :, 8:9]
X_val_ecg = X_val[:, :, 9:10]

X_test_mpu = X_test[:, :, :6]
X_test_max = X_test[:, :, 6:8]
X_test_emg = X_test[:, :, 8:9]
X_test_ecg = X_test[:, :, 9:10]

# Train classifier
model = build_model(window_size, 10)
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(np.argmax(y_train, axis=1)),
                                                  y=np.argmax(y_train, axis=1))
class_weights_dict = dict(enumerate(class_weights))

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

history = model.fit(
    [X_train_mpu, X_train_max, X_train_emg, X_train_ecg], y_train,
    validation_data=([X_val_mpu, X_val_max, X_val_emg, X_val_ecg], y_val),
    epochs=100,
    batch_size=64,
    class_weight=class_weights_dict,
    callbacks=[lr_scheduler, early_stopping],
    verbose=1
)

test_loss, test_accuracy = model.evaluate(
    [X_test_mpu, X_test_max, X_test_emg, X_test_ecg], y_test, verbose=0
)
print(f"Test Accuracy on Simulated Data: {test_accuracy:.4f}")

# Step 4: Autoencoder for Anomaly Detection
def build_autoencoder(window_size, num_features):
    input_layer = layers.Input(shape=(window_size, num_features))
    x = layers.LSTM(128, return_sequences=True)(input_layer)
    x = layers.LSTM(64)(x)
    x = layers.RepeatVector(window_size)(x)
    x = layers.LSTM(64, return_sequences=True)(x)
    x = layers.LSTM(128, return_sequences=True)(x)
    output_layer = layers.TimeDistributed(layers.Dense(num_features))(x)
    return Model(input_layer, output_layer)

X_train_normal = X_train[np.argmax(y_train, axis=1) == 0]
autoencoder = build_autoencoder(window_size, 10)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_train_normal, X_train_normal, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

# Step 5: Function to Process User Sensor Input
def process_user_input(scaler, model, autoencoder, window_size):
    print("\n=== Real-Time Stroke Detection ===")
    print("Enter sensor data (accel_x, accel_y, accel_z, gyro_x, gyro_y, gyro_z, heart_rate, spo2, emg_signal, ecg_signal)")
    print(f"Requires {window_size} samples at 100 Hz (5 seconds). Enter one sample per line, or 'done' to finish early.")

    sensor_data = []
    sensor_cols = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z',
                   'heart_rate', 'spo2', 'emg_signal', 'ecg_signal']

    for i in range(window_size):
        user_input = input(f"Sample {i+1}/{window_size}: ")
        if user_input.lower() == 'done':
            break
        try:
            values = [float(x) for x in user_input.split(',')]
            if len(values) != 10:
                print("Error: Enter exactly 10 values separated by commas.")
                continue
            sensor_data.append(values)
        except ValueError:
            print("Error: Invalid input. Use numbers separated by commas (e.g., 1.2, -0.5, ...)")
            continue

    # Pad with zeros or truncate to window_size
    if len(sensor_data) < window_size:
        sensor_data.extend([[0] * 10 for _ in range(window_size - len(sensor_data))])
    sensor_data = sensor_data[:window_size]

    # Convert to DataFrame and normalize
    df_user = pd.DataFrame(sensor_data, columns=sensor_cols)
    X_user = scaler.transform(df_user[sensor_cols])
    X_user = np.array([X_user])  # Shape: (1, window_size, 10)

    # Split for multi-input model
    X_user_mpu = X_user[:, :, :6]
    X_user_max = X_user[:, :, 6:8]
    X_user_emg = X_user[:, :, 8:9]
    X_user_ecg = X_user[:, :, 9:10]

    # Classifier prediction
    pred = model.predict([X_user_mpu, X_user_max, X_user_emg, X_user_ecg])
    class_pred = np.argmax(pred, axis=1)[0]
    class_labels = {0: "Normal", 1: "True Stroke", 2: "Fake Stroke"}
    print(f"Classification: {class_labels[class_pred]} (Confidence: {pred[0][class_pred]:.4f})")

    # Autoencoder anomaly detection
    reconstruction = autoencoder.predict(X_user)
    mse = np.mean(np.square(X_user - reconstruction))
    threshold = np.percentile(autoencoder.predict(X_train_normal).mean(axis=(1, 2)), 95)  # Precomputed threshold
    if mse > threshold:
        if class_pred == 1:
            print("Pre-Stroke Warning: True Stroke Likely")
        elif class_pred == 2:
            print("Pre-Stroke Warning: Possible Fake Stroke")
        else:
            print("Pre-Stroke Warning: Anomaly Detected")
    else:
        print("No Pre-Stroke Warning")

    return class_pred, mse

# Step 6: Test with User Input
while True:
    process_user_input(scaler, model, autoencoder, window_size)
    cont = input("\nContinue testing? (yes/no): ")
    if cont.lower() != 'yes':
        break

print("Testing complete.")

Windowed data shape: (79, 500, 10) (79,)
Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 25s 25s/step - accuracy: 0.3830 - loss: 1.1158 - val_accuracy: 0.9375 - val_loss: 0.9829 - learning_rate: 5.0000e-04
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.7021 - loss: 0.8596 - val_accuracy: 0.9375 - val_loss: 0.8724 - learning_rate: 5.0000e-04
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.8511 - loss: 0.9164 - val_accuracy: 0.9375 - val_loss: 0.8084 - learning_rate: 5.0000e-04
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - accuracy: 0.7234 - loss: 1.0264 - val_accuracy: 0.9375 - val_loss: 0.7891 - learning_rate: 5.0000e-04
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - accuracy: 0.7021 - loss: 0.8210 - val_accuracy: 0.9375 - val_loss: 0.7698 - learning_rate: 5.0000e-04
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.6383 - loss: 0.9081 - val_accuracy: 0.9375 - val_loss: 0.7688 - learning_rate: 5.0000e-04
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9

KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install numpy pandas scikit-learn tensorflow imbalanced-learn
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Load dataset
df = pd.read_csv("/content/fall_detection_dataset_10000.csv")

# Define features and target
X = df.drop(columns=["fall_detected"]).values  # Features (sensor data)
y = df["fall_detected"].values  # Target (1 = Real Fall, 0 = False Detection)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Handle imbalanced data using SMOTE
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# Reshape for LSTM input
X_resampled = X_resampled.reshape(X_resampled.shape[0], X_resampled.shape[1], 1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
# Create LSTM-based model
model = Sequential()

# Input Layer
model.add(Bidirectional(LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], 1))))
model.add(Dropout(0.3))

# Hidden Layers
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.2))

# Output Layer
model.add(Dense(1, activation="sigmoid"))

# Compile Model
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 46s 76ms/step - accuracy: 0.4996 - loss: 0.6936 - val_accuracy: 0.5106 - val_loss: 0.6930
Epoch 2/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 43s 83ms/step - accuracy: 0.5064 - loss: 0.6935 - val_accuracy: 0.5052 - val_loss: 0.6917
Epoch 3/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 39s 78ms/step - accuracy: 0.5242 - loss: 0.6912 - val_accuracy: 0.5361 - val_loss: 0.6896
Epoch 4/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 28s 80ms/step - accuracy: 0.5367 - loss: 0.6895 - val_accuracy: 0.5239 - val_loss: 0.6902
Epoch 5/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 27s 76ms/step - accuracy: 0.5374 - loss: 0.6887 - val_accuracy: 0.5332 - val_loss: 0.6907
Epoch 6/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 41s 77ms/step - accuracy: 0.5516 - loss: 0.6865 - val_accuracy: 0.5389 - val_loss: 0.6886
Epoch 7/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 40s 73ms/step - accuracy: 0.5486 - loss: 0.6871 - val_accuracy: 0.5274 - val_loss: 0.6894
Epoch 8/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 45s 84ms/step - accuracy: 0.5483 - loss: 0.6856 - 

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Flatten, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# ✅ Load dataset
df = pd.read_csv("/content/fall_detection_dataset_10000.csv")

# ✅ Ensure correct feature selection
features = ["acc_x", "acc_y", "acc_z", "emg_signal", "heart_rate_ecg", "spo2"]
target = "fall_detected"  # 1 = Fall, 0 = No Fall, 2 = False Fall

# Check if all required features exist
missing_features = [col for col in features if col not in df.columns]
if missing_features:
    raise ValueError(f"Missing features: {missing_features}")

# ✅ Split features and labels
X = df[features].values
y = df[target].values

# ✅ Normalize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ✅ Handle class imbalance using SMOTE
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# ✅ Reshape data for CNN/LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# ✅ Define Model: CNN + LSTM for Temporal & Spatial Features
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    BatchNormalization(),
    Dropout(0.3),

    LSTM(128, return_sequences=True),
    Dropout(0.4),

    LSTM(64),
    Dropout(0.3),

    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes: Fall, No Fall, False Fall
])

# ✅ Compile Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# ✅ Train Model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# ✅ Evaluate Model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"✅ Final Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


349/349 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.5029 - loss: 0.7810 - val_accuracy: 0.5091 - val_loss: 0.6954
Epoch 2/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.5117 - loss: 0.6977 - val_accuracy: 0.5221 - val_loss: 0.6922
Epoch 3/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.5227 - loss: 0.6937 - val_accuracy: 0.5077 - val_loss: 0.6921
Epoch 4/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.5128 - loss: 0.6940 - val_accuracy: 0.5142 - val_loss: 0.6929
Epoch 5/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.5205 - loss: 0.6923 - val_accuracy: 0.5389 - val_loss: 0.6906
Epoch 6/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.5243 - loss: 0.6918 - val_accuracy: 0.5149 - val_loss: 0.6933
Epoch 7/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.5275 - loss: 0.6916 - val_accuracy: 0.5307 - val_loss: 0.6904
Epoch 8/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.5408 - loss: 0.6873 - val_accuracy: 0.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Flatten, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# ✅ Load dataset
df = pd.read_csv("/content/fall_detection_dataset_10000.csv")

# ✅ Ensure correct feature selection
features = ["acc_x", "acc_y", "acc_z", "gyro_x","gyro_y","gyro_z","emg_signal", "heart_rate_ecg", "spo2","ecg_variability"]
target = "fall_detected"  # 1 = Fall, 0 = No Fall, 2 = False Fall

# Check if all required features exist
missing_features = [col for col in features if col not in df.columns]
if missing_features:
    raise ValueError(f"Missing features: {missing_features}")

# ✅ Split features and labels
X = df[features].values
y = df[target].values

# ✅ Normalize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ✅ Handle class imbalance using SMOTE
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# ✅ Reshape data for CNN/LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# ✅ Define Model: CNN + LSTM for Temporal & Spatial Features
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    BatchNormalization(),
    Dropout(0.3),

    LSTM(128, return_sequences=True),
    Dropout(0.4),

    LSTM(64),
    Dropout(0.3),

    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes: Fall, No Fall, False Fall
])

# ✅ Compile Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# ✅ Train Model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# ✅ Evaluate Model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"✅ Final Test Accuracy: {test_accuracy * 100:.2f}%")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 15s 28ms/step - accuracy: 0.5087 - loss: 0.7809 - val_accuracy: 0.5045 - val_loss: 0.6964
Epoch 2/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - accuracy: 0.5193 - loss: 0.6965 - val_accuracy: 0.5242 - val_loss: 0.6903
Epoch 3/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - accuracy: 0.5205 - loss: 0.6936 - val_accuracy: 0.5260 - val_loss: 0.6902
Epoch 4/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 21s 60ms/step - accuracy: 0.5244 - loss: 0.6925 - val_accuracy: 0.5278 - val_loss: 0.6962
Epoch 5/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 30s 28ms/step - accuracy: 0.5530 - loss: 0.6880 - val_accuracy: 0.5414 - val_loss: 0.6901
Epoch 6/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - accuracy: 0.5444 - loss: 0.6881 - val_accuracy: 0.5274 - val_loss: 0.6896
Epoch 7/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.5333 - loss: 0.6893 - val_accuracy: 0.5501 - val_loss: 0.6848
Epoch 8/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.5539 - loss: 0.6848 - val

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Flatten, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# ✅ Load dataset
df = pd.read_csv("/content/fall_detection_dataset_10000.csv")

# ✅ Ensure correct feature selection
features = ["acc_x", "acc_y", "acc_z", "gyro_x","gyro_y","gyro_z","emg_signal", "heart_rate_ecg", "spo2","ecg_variability","heart_rate_spo2"]
target = "fall_detected"  # 1 = Fall, 0 = No Fall, 2 = False Fall

# Check if all required features exist
missing_features = [col for col in features if col not in df.columns]
if missing_features:
    raise ValueError(f"Missing features: {missing_features}")

# ✅ Split features and labels
X = df[features].values
y = df[target].values

# ✅ Normalize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ✅ Handle class imbalance using SMOTE
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# ✅ Reshape data for CNN/LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# ✅ Define Model: CNN + LSTM for Temporal & Spatial Features
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    BatchNormalization(),
    Dropout(0.3),

    LSTM(128, return_sequences=True),
    Dropout(0.4),

    LSTM(64),
    Dropout(0.3),

    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes: Fall, No Fall, False Fall
])

# ✅ Compile Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# ✅ Train Model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# ✅ Evaluate Model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"✅ Final Test Accuracy: {test_accuracy * 100:.2f}%")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 20s 32ms/step - accuracy: 0.5001 - loss: 0.7806 - val_accuracy: 0.5303 - val_loss: 0.6932
Epoch 2/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 0.5241 - loss: 0.6952 - val_accuracy: 0.5289 - val_loss: 0.6919
Epoch 3/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - accuracy: 0.5314 - loss: 0.6918 - val_accuracy: 0.5307 - val_loss: 0.6888
Epoch 4/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 19s 27ms/step - accuracy: 0.5417 - loss: 0.6875 - val_accuracy: 0.5396 - val_loss: 0.6873
Epoch 5/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - accuracy: 0.5462 - loss: 0.6864 - val_accuracy: 0.5346 - val_loss: 0.6904
Epoch 6/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 20s 30ms/step - accuracy: 0.5506 - loss: 0.6852 - val_accuracy: 0.5411 - val_loss: 0.6874
Epoch 7/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 19s 25ms/step - accuracy: 0.5669 - loss: 0.6833 - val_accuracy: 0.5536 - val_loss: 0.6858
Epoch 8/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - accuracy: 0.5717 - loss: 0.6776 - 

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Conv1D, Flatten, Dropout, BatchNormalization, Bidirectional, Attention, Permute, Multiply
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# ✅ Load dataset
df = pd.read_csv("/content/fall_detection_dataset_10000.csv")

# ✅ Ensure correct feature selection
features = ["acc_x", "acc_y", "acc_z", "gyro_x","gyro_y","gyro_z","emg_signal", "heart_rate_ecg", "spo2","ecg_variability","heart_rate_spo2"]
target = "fall_detected"  # 1 = Fall, 0 = No Fall, 2 = False Fall

# ✅ Check if features exist
missing_features = [col for col in features if col not in df.columns]
if missing_features:
    raise ValueError(f"Missing features: {missing_features}")

# ✅ Prepare data
X = df[features].values
y = df[target].values

# ✅ Normalize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ✅ Handle class imbalance using SMOTE
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# ✅ Reshape for CNN-LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# ✅ Define Model: CNN + BiLSTM + Fixed Attention Layer
input_layer = Input(shape=(X_train.shape[1], 1))

# 🟢 Convolutional Layers
x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# 🔵 BiLSTM for capturing temporal patterns
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Dropout(0.4)(x)

# 🟠 Fixed Attention Mechanism
attention_scores = Dense(1, activation="tanh")(x)  # Compute attention scores
attention_scores = Flatten()(attention_scores)
attention_scores = tf.keras.activations.softmax(attention_scores)  # Normalize scores
attention_scores = tf.keras.layers.RepeatVector(128)(attention_scores)  # Match LSTM output shape
attention_scores = Permute((2, 1))(attention_scores)
x = Multiply()([x, attention_scores])  # Apply attention weights
x = Flatten()(x)  # Flatten for fully connected layer

# 🔴 Fully Connected Layers
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
output_layer = Dense(3, activation='softmax')(x)  # 3 classes: Fall, No Fall, False Fall

# ✅ Compile Model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# ✅ Train Model
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test))

# ✅ Evaluate Model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"✅ Final Test Accuracy: {test_accuracy * 100:.2f}%")


ValueError: Inputs have incompatible shapes. Received shapes (11, 256) and (11, 128)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Conv1D, Flatten, Dropout, BatchNormalization, Bidirectional, Attention, GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# ✅ Load dataset
df = pd.read_csv("/content/fall_detection_dataset_10000.csv")

# ✅ Ensure correct feature selection
features = ["acc_x", "acc_y", "acc_z", "gyro_x","gyro_y","gyro_z","emg_signal", "heart_rate_ecg", "spo2","ecg_variability","heart_rate_spo2"]
target = "fall_detected"  # 1 = Fall, 0 = No Fall, 2 = False Fall

# ✅ Check if features exist
missing_features = [col for col in features if col not in df.columns]
if missing_features:
    raise ValueError(f"Missing features: {missing_features}")

# ✅ Prepare data
X = df[features].values
y = df[target].values

# ✅ Normalize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ✅ Handle class imbalance using SMOTE
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# ✅ Reshape for CNN-LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# ✅ Define Model: CNN + BiLSTM + Fixed Attention
input_layer = Input(shape=(X_train.shape[1], 1))

# 🟢 Convolutional Layers
x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# 🔵 BiLSTM for capturing temporal patterns
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Dropout(0.4)(x)

# 🟠 Fixed Attention Mechanism
attention_layer = Attention()([x, x])  # Self-attention
x = GlobalAveragePooling1D()(attention_layer)  # Compress features

# 🔴 Fully Connected Layers
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
output_layer = Dense(3, activation='softmax')(x)  # 3 classes: Fall, No Fall, False Fall

# ✅ Compile Model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# ✅ Train Model
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test))

# ✅ Evaluate Model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"✅ Final Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 20s 74ms/step - accuracy: 0.5062 - loss: 0.8279 - val_accuracy: 0.5181 - val_loss: 0.8243
Epoch 2/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 20s 74ms/step - accuracy: 0.5204 - loss: 0.7077 - val_accuracy: 0.5267 - val_loss: 0.7077
Epoch 3/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 21s 76ms/step - accuracy: 0.5410 - loss: 0.6942 - val_accuracy: 0.5350 - val_loss: 0.6888
Epoch 4/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 68ms/step - accuracy: 0.5432 - loss: 0.6893 - val_accuracy: 0.5389 - val_loss: 0.6880
Epoch 5/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 12s 68ms/step - accuracy: 0.5286 - loss: 0.6902 - val_accuracy: 0.5310 - val_loss: 0.6902
Epoch 6/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 21s 69ms/step - accuracy: 0.5498 - loss: 0.6876 - val_accuracy: 0.5414 - val_loss: 0.6880
Epoch 7/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 21s 71ms/step - accuracy: 0.5511 - loss: 0.6869 - val_accuracy: 0.5260 - val_loss: 0.6869
Epoch 8/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 20s 70ms/step - accuracy: 0.5469 - loss: 0

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# ✅ Load dataset
file_path = "fall_detection_dataset_10000.csv"  # Update path if needed
df = pd.read_csv(file_path)

# ✅ Check for missing values
df.dropna(inplace=True)

# ✅ Split features and labels
X = df.drop(columns=["fall_type"]).values  # All sensor readings
y = df["fall_type"].values  # 0 = False Fall, 1 = Real Fall

# ✅ Normalize Data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ✅ Reshape for LSTM (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# ✅ Build CNN + LSTM Model
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),

    LSTM(128, return_sequences=True),
    Dropout(0.3),

    LSTM(64, return_sequences=False),
    Dropout(0.3),

    Dense(32, activation='relu'),
    Dropout(0.2),

    Dense(1, activation='sigmoid')  # Binary Classification (Fall vs False Fall)
])

# ✅ Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ✅ Train Model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

# ✅ Evaluate Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# ✅ Save Model
model.save("fall_detection_model.h5")


KeyError: "['fall_type'] not found in axis"